<a href="https://colab.research.google.com/github/sw-programmer/Coursework1/blob/sangwoo/CW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Coursework 1**

1.   ### **Computationally Efficient Eigenfaces**
2.   ### **Incremental PCA**
3.   ### **PCA-LDA for Face Recognition**
4.   ### **K-means codebook**
5.   ### **RF classifier**


**by 20190223	Soohyun Ryu,	20190247	Sangwoo Park**






**1. Setting!**

In [23]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#mat_file = io.loadmat("/content/drive/MyDrive/CV_ML/CW1/face.mat")
mat_file = io.loadmat("./face.mat")
mat_X = mat_file['X']
mat_Y = mat_file['l'][0]

print(mat_file.keys()) 
print(mat_X.shape)                         # 총 52명. 각 10장의 사진. 

dict_keys(['__header__', '__version__', '__globals__', 'X', 'l'])
(2576, 520)


In [ ]:
df = pd.DataFrame(data=mat_X).transpose()
df['label'] = mat_Y

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values                   # column 'Target'

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    stratify=Y, 
                                                    random_state=1000)          
# 얼굴 그룹 비율 유지               
# test/train ratio = 0.2
# random_state -> before dividing, seed for shuffling
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
np.unique(y_train, return_counts=True)    # 52명의 얼굴이 80%씩 고르게 train set에 들어간 모습

## **2. Incremental PCA**



In [32]:
# 평균 얼굴 계산 함수 
def AvgCompute(X): 
  Xbar = np.mean(X, axis=0)  
  matrix_A = X - Xbar 
  return matrix_A, Xbar

# 고유값, 고유벡터 계산 함수
def eig(S): 
  eig_vals, eig_vecs = np.linalg.eig(S) 
  sort_indices = np.argsort(eig_vals)[::-1] # 내림차순 정렬 
  return eig_vals[sort_indices], eig_vecs[:, sort_indices]

# Reconstruction에 쓰일 함수
def reconstruct(X, PC): 
  return (X @ PC) @ PC.T

# PCA 과정
def PCA(X, num_components, low_dim = False): # X == train set, num_components == 선택할 eigen vector 개수
  start = time.time()
  N, D = X.shape 
  matrix_A, X_bar = AvgCompute(X)
  if not low_dim:
    S = (matrix_A.T @ matrix_A) / N
  else:
    S = (matrix_A @ matrix_A.T) / N

  eig_vals, eig_vecs = eig(S)
  principal_vals, principal_components = np.real(eig_vals[:num_components]), np.real(eig_vecs[:,:num_components]) 
  
  if low_dim:
    principal_components = matrix_A.T @ principal_components

  reconst_X = reconstruct(matrix_A, principal_components) + X_bar 
  comp_time = time.time() - start
  
  return reconst_X, principal_components, comp_time

Batch PCA

In [ ]:
base_num = 100

Batch_PCA_X, Batch_PCA_pc, Batch_PCA_time = PCA(X_train, base_num, False)
Batch_PCA_recon_error = np.linalg.norm((X_train - Batch_PCA_X), None)

print(f"Training time: {Batch_PCA_time}sec")
print(f"Reconstruction error: {Batch_PCA_recon_error}")

Incremental PCA

In [ ]:
def PCA_decompose(X, num_components, low_dim = False):
  start = time.time()

  N, D = X.shape 
  matrix_A, X_bar = AvgCompute(X)
  X_bar = np.reshape(X_bar, X_bar.shape + (1,))
  if not low_dim:
    S = (matrix_A.T @ matrix_A) / N
  else:
    S = (matrix_A @ matrix_A.T) / N

  eig_vals, eig_vecs = eig(S)
  principal_vals, principal_components = np.real(eig_vals[:num_components]), np.real(eig_vecs[:,:num_components]) 

  if low_dim:
    principal_components = matrix_A.T @ principal_components

  reconst_X = reconstruct(matrix_A, principal_components) + X_bar 
  comp_time = time.time() - start
  
  return X_bar, N, S, principal_components, comp_time, reconst_X

In [ ]:
https://gist.github.com/iizukak/1287876/edad3c337844fac34f7e56ec09f9cb27d4907cc7# 

def gs_cofficient(v1, v2):
    return np.dot(v2, v1) / np.dot(v1, v1)

def multiply(cofficient, v):
    return map((lambda x : x * cofficient), v)

def proj(v1, v2):
    return multiply(gs_cofficient(v1, v2) , v1)

def gram_schmidt(X):
    result = []
    for i in range(len(X)):
        temp_vec = X[i]
        for w in result:
            proj_vec = proj(w, X[i])
            temp_vec = map(lambda x, y : x - y, temp_vec, proj_vec)
        result.append(temp_vec)
    for w in result:
        if np.array_equal(np.zeros_like(w), w):
            result.remove(w)
    return result

In [ ]:
def merge_PCAs(M1, N1, S1, P1, M2, N2, S2, P2, num_components, X):
    N3 = N1 + N2
    M3 = (N1*M1 + N2*M2) / N3
    S3 = (N1/N3) * S1 + (N2/N3) * S2 + (N1*N2/N3) * (M1-M2)@(M1-M2).T
    X = []
    for i in range(len(P1.shape[0])):
        X.append(P1[:,i])
    for i in range(len(P2.shape[0])):
        X.append(P2[:,i])
    X.append(M1-M2)
    
    suff_spanning = gram_schmidt(X)
    suff_spanning = np.concatenate(suff_spanning, axis=0).T

    new_mat = suff_spanning.T @ S3 @ suff_spanning

    # ignore all other computations
    start = time.time()
    _, eig_vecs = eig(new_mat)
    comp_time = time.time()-start

    _, R = np.real(eig_vecs[:,:num_components])

    P3 = suff_spanning @ R

    reconst_X = reconstruct((X-M3), P3) + M3 

    return M3, N3, S3, P3, comp_time, reconst_X

In [ ]:
# split train dataset to 4 batches
X_train11, X_train22, y_train11, y_train22 = train_test_split(X_train, 
                                                              y_train, 
                                                              test_size=0.5, 
                                                              shuffle=True,
                                                              stratify=Y, 
                                                              random_state=1000) 

X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train11, 
                                                          y_train11, 
                                                          test_size=0.5, 
                                                          shuffle=True,
                                                          stratify=Y, 
                                                          random_state=1000)

X_train3, X_train4, y_train3, y_train4 = train_test_split(X_train, 
                                                          y_train, 
                                                          test_size=0.5, 
                                                          shuffle=True,
                                                          stratify=Y, 
                                                          random_state=1000)  

In [ ]:
X_train1_2 = np.concatenate((X_train1, X_train2), axis=0)
X_train1_2_3 = np.concatenate((X_train1_2, X_train3), axis=0)

In [ ]:
M1, N1, S1, P1, comp_time1, reconst_X1 = PCA_decompose(X_train1, base_num)
M2, N2, S2, P2, comp_time2, reconst_X2 = PCA_decompose(X_train2, base_num)
M3, N3, S3, P3, comp_time3, reconst_X3 = PCA_decompose(X_train3, base_num)
M4, N4, S4, P4, comp_time4, reconst_X4 = PCA_decompose(X_train4, base_num)

M5, N5, S5, P5, comp_time5, reconst_X5 = merge_PCAs(M1, N1, S1, P1, M2, N2, S2, P2, base_num, X_train1_2)
M6, N6, S6, P6, comp_time6, reconst_X6 = merge_PCAs(M5, N5, S5, P5, M3, N3, S3, P3, base_num, X_train1_2_3)
M7, N7, S7, P7, comp_time7, reconst_X7 = merge_PCAs(M6, N6, S6, P6, M4, N4, S4, P4, base_num, X_train)

print(f"Training time: {comp_time1+comp_time2+comp_time3+comp_time4+comp_time5+comp_time6+comp_time7}")

first_subset_PCA_recon_error = np.linalg.norm((X_train1 - reconst_X1), None)
print(f"Reconstruction error _ first subset: {first_subset_PCA_recon_error}")
Incremental_PCA_recon_error = np.linalg.norm((X_train - reconst_X7), None)
print(f"Reconstruction error _ first subset: {Incremental_PCA_recon_error}")

NN Classifier

In [ ]:
from sklearn.metrics import accuracy_score

# 너꺼 그대로 복붙했는데 변수이름 바꿔야 할듯!
# NN Classfication
X_result = X_test @ principal_components

y_pred = []
label = 0
for query in X_result:
  Args = []
  for trained in X_Projected:
    Args.append(np.linalg.norm(query - trained))
  y_pred.append(y_train[np.argmin(Args)])

print(accuracy_score(y_test, y_pred))